# <span style="color: blue">Cours et exercices de la découverte de Coq</span>

## <span style="color: blue">__Un langage fonctionnel pur (et même plus)__</span>

In [ ]:
Module nat_bool. (* Pour ne pas masquer les lemmes et définitions de la bibliothèque 
standard sur nat et bool, nous ouvrons un module. *)

### __Type algébrique de données__

• Un type défini par les différents cas possibles, les constructeurs

• Les constructeurs sont des objets à part entière

• Existe en OCaml, ≈ type enum en C

Voici un exemple :

In [ ]:
Inductive route : Type :=
| departementale : route
| nationale : route
| autoroute : route.

In [ ]:
Check route. (* route : Type *)

In [ ]:
Check autoroute. (* autoroute : route *)

Pour examiner un terme dont le type est inductif, on utilise le
__pattern-matching__ (filtrage).

In [ ]:
Definition agrandir (r : route) := 
  match r with
  | departementale => nationale
  | nationale => autoroute
  | autoroute => autoroute
  end.

In [ ]:
Check agrandir. (* agrandir : route -> route *)

In [ ]:
Eval compute in (agrandir (agrandir nationale)).

In [ ]:
Les constructeurs peuvent prendre des arguments.

In [ ]:
Inductive terrain : Type :=
| t_terre : terrain
| t_route : route -> terrain
| t_batiment : terrain.

In [ ]:
Check (t_route nationale). (* terrain *)

In [ ]:
Definition agrandir_terrain (t : terrain) := match t with
| t_terre    => t_batiment
| t_route r  => t_route (agrandir r)
| t_batiment => t_batiment
end.

In [ ]:
Check agrandir_terrain. (* : terrain -> terrain *)

A retenir ! - <span style="color:red">__Contraintes sur le pattern-matching__ : Exhaustif et non-redondant. </span>

### __Des types plus communs__

Booléens - le type bool est prédéfini de la façon suivante :

In [ ]:
Inductive bool : Type :=
| true : bool
| false : bool.

In [ ]:
Check true. (* true : bool *)

In [ ]:
Check false. (* false : bool *)

In [ ]:
Print bool.

In [ ]:
Definition negb b :=
match b with
| true  => false
| false => true
end.

In [ ]:
Check negb. (* negb : bool → bool *)

A vous de jouer !

Exercice 1. : Définir les fonctions [andb] et [orb] sur les boolénns.

In [ ]:
Definition andb b1 b2 :=
  match b1 with
  | true  => b2
  | false => false
  end.

In [ ]:
Definition orb b1 b2 :=
  match b1 with
  | true  => true
  | false => b2
  end.

On peut définir des types récursifs, comme les entiers de Peano :

In [ ]:
Inductive nat : Type :=
| O : nat        (* lettre O, interprété comme 0 *)
| S : nat -> nat (* successeur *).

In [ ]:
Check O. (* O : nat *)

In [ ]:
Check S. (* S : nat -> nat *)

In [ ]:
Notation "0" := O.
Notation "1" := (S O).

Les listes de booléens

In [ ]:
Inductive list_bool : Type :=
| nilb : list_bool
| consb : bool -> list_bool -> list_bool.

In [ ]:
Check nilb. (* nil : list_bool *)

In [ ]:
Check consb. (* cons : bool -> list_bool -> list_bool *)

In [ ]:
Definition example := consb true (consb false (consb true nilb)).

In [ ]:
Check example.

### __Fonctions récursives__

On peut définir des fonctions récursives sur des types récursifs.

In [ ]:
Fixpoint plus n m :=
match n with
| O => m
| S n’ => S (plus n’ m)
end.
(*
plus is defined
plus is recursively defined (decreasing on 1st argument)
*)


In [ ]:
Eval compute in (plus 3 4)

In [ ]:
Notation "n + m" := (plus n m).

In [ ]:
Eval compute in (0 + 1).

A retenir - <span style="color:red"> __Terminaison__ : 
La fonction doit terminer ! </span>
    
<span style="color:red"> Cas détecté automatiquement par Coq : récurrence structurelle </span>

Concaténation de listes de booléens

In [7]:
Fixpoint appb l1 l2 :=
match l1 with
| nilb => l2
| consb a l => consb a (appb l l2)
end.
(* appb is recursively defined (decreasing on 1st argument) *)

SyntaxError: invalid syntax (<ipython-input-7-38d048c4e9c4>, line 1)

In [ ]:
Check appb. (* list_bool -> list_bool -> list_bool *)

Alors que si vous lancez cet exemple, Coq ne l'accepte pas !

In [ ]:
(* Fixpoint appb_inc l1 l2 :=
  match l1 with
  | nilb => l2
  | consb a l => consb a (appb_inc l1 l2)
    end. 
*)
(* Error: Cannot guess decreasing argument of fix. *)

### Autres remarques sur les fonctions

• Fonctions comme valeurs de première classe

In [ ]:
Definition apply_neg (f:_-> _-> bool) b1 b2 :=
f (negb b1) (negb b2).

In [ ]:
Check apply_neg. 
(* apply_neg : (bool -> bool -> bool) -> bool -> bool -> bool *)

• Applications partielles

In [ ]:
Definition nor := apply_neg andb.

In [ ]:
Check nor. (* nor : bool -> bool -> bool *)

• Fonctions anonymes

In [ ]:
Definition norb := 
   apply_neg (fun b1 => fun b2 => b2).

In [ ]:
Eval compute in (norb true false).

In [ ]:
Definition idb : bool -> bool := fun b => b.
(* identique à
 Definition idb (b : bool) := b.
*)

In [ ]:
Definition my_first := apply_neg (fun b1 => fun b2 => b1).

In [ ]:
Eval compute in (my_first true false).

### Type polymorphique

Les types peuvent être paramétrés par d’autres types.

Exemple le plus classique : les listes.

In [ ]:
Inductive list (A:Type) : Type :=
| nil : list A
| cons : A -> list A -> list A.

In [ ]:
Check list. (* list : Type -> Type *)

In [ ]:
Check nil. (* nil : forall A : Type, list A *)

In [ ]:
Check cons. (* cons : forall A : Type, A -> list A -> list A *)

In [ ]:
Eval compute in (cons nat 1 (cons nat (S 1) (nil nat))).

In [ ]:
Eval compute in (cons bool true (cons bool false (nil bool))).

### Fonctions polymorphiques

Les fonctions peuvent aussi être paramétrés par des types.

Un exemple un peu artificiel.

In [ ]:
Definition id (A:Type) (x:A) := x.

In [ ]:
Check id. (* id : forall A : Type, A -> A *)

Un exemple sur les listes.

In [ ]:
Fixpoint length (A:Type) (l:list A) :=
  match l with
  | nil _ => 0
  | cons _ _ t => S (length A t)
  end. 

In [ ]:
Eval compute in (length bool 
   (cons bool true 
         (cons bool false (nil bool)))).

In [ ]:
Fixpoint app (A:Type) l1 l2 :=
  match l1 with
  | nil _      => l2
  | cons _ a l => cons A a (app A l l2)
    end. 

Retour sur un ancien exemple :

In [ ]:
Definition apply_neg (f:_-> _-> bool) b1 b2 :=
 f (negb b1) (negb b2).

Pourquoi se limiter à bool ?

In [ ]:
Definition apply_neg_gen (A:Type) (f:_->_->A) b1 b2 :=
  f (negb b1) (negb b2).

In [ ]:
Definition nor2 := apply_neg_gen bool  andb.
Check nor2. 
(* nor2 : forall A : Type, (bool -> bool -> A) -> bool -> bool -> A *)

In [ ]:
Definition toto (b1 b2 : bool) := 0. 

Eval compute in apply_neg_gen  nat toto true false.

### Lisibilité

On peut améliorer la lisibilité des termes manipulés.

• Avec des paramètres implicites

Si un argument peut toujours être inféré à partir d’un autre,
on peut le rendre implicite.

In [ ]:
Definition apply_neg_gen2 {A:Type} (f:_->_-> A) b1 b2 :=
  f (negb b1) (negb b2).

In [ ]:
Definition nor3 := apply_neg_gen2 andb.
(* au lieu de apply_neg_gen bool andb
ou apply_neg_gen _ andb *)

In [ ]:
• Avec des notations

In [ ]:
Notation "x = y" := ...

In [ ]:
Notation "[ ]" := nil.

## <span style="color:blue"> Logique </span>

Précédemment, tous nos objets étaient dans Type. 

Les énoncés sont dans un autre type : <span style="color:red"> Prop </span>

In [ ]:
Check True. (* True : Prop *)

In [ ]:
Check True -> False. (* True -> False : Prop *)

__Énoncé d’un théorème__

• Un premier théorème

In [ ]:
Theorem negb_true_false : negb true = false.

• Plus intéressant ?

In [ ]:
Theorem app_nil_l : forall {A : Type} (l : list A), app [] l = l.

• Mais peut-on exprimer autre chose que des égalités ?

__Constantes, connecteurs, quantificateurs__

• Constantes : True, False

• Connecteurs : ~ (non), ∧ (et), ∨ (ou), → (implique)

• Quantificateurs : forall, exists

 A retenir ! - <span style="color:red"> Logique intuitionniste : p → q n'est pas équivalent à : ~ p ∨ q </span>

• Logique d’ordre supérieur : on peut quantifier sur des
fonctions et des prédicats

In [ ]:
Lemma bool_trivial : forall (P : bool -> Prop) b, P b -> P b.
Admitted.

__Remarque importante__ :
Comme pour les fonctions qui reçoivent leurs arguments un par un
et non un couple, les énoncés à plusieurs hypothèses utilisent
plusieurs implications, plutôt que la conjonction.

In [ ]:
Lemma bool_ext : forall (P : bool -> Prop), P true /\ P false ->
  forall b, P b.
Admitted.

In [ ]:
Lemma bool_ext2 : forall (P : bool -> Prop), P true -> P false ->
  forall b, P b.
Admitted.

### Les prédicats inductifs

On a vu comment définir des types inductifs. De la même manière,
on peut définir des prédicats inductifs (à la Prolog).

Un peu de patience ....

## <span style="color:blue"> Preuve </span>

Comment écrire la preuve d’un théorème ?

• De manière interactive !

• En s’appuyant sur un langage de tactiques permettant de
transformer successivement le but actuel en un ou des buts plus simples à résoudre

• Certaines tactiques de base ont une correspondance évidente dans le calcul des séquents ou en déduction naturelle (par ex. intros, split)

• L’utilisateur peut définir des tactiques (par exemple une tactique qui enchaîne plusieurs tactiques de base)

Prenons un exemple !

In [ ]:
Lemma andb_prop : forall b1 b2,
  andb b1 b2 = true -> b1 = true /\ b2 = true.
Proof.

In [ ]:
intros b1 b2 H. (* introduit les hypothèses *)

In [ ]:
split. (* sépare le but en deux sous-buts *)

In [ ]:
  - destruct b1. (* raisonnement par cas *)

In [ ]:
    + reflexivity. (* true = true *)

In [ ]:
    + simpl in H. discriminate H. (* false <> true *)

In [ ]:
  - destruct b2. (* raisonnement par cas *)

In [ ]:
    + reflexivity. (* true = true *)

In [ ]:
    + destruct b1 .  (* raisonnement par cas *)

In [ ]:
      * simpl in H. discriminate H. (* false <> true *)

In [ ]:
      * simpl in H. discriminate H. (* false <> true *)

In [ ]:
Qed.

Petite optimisation :

In [ ]:
Lemma andb_prop : forall b1 b2,
andb b1 b2 = true -> b1 = true ∧ b2 = true.
Proof.
intros b1 b2 H. (* introduit les hypothèses *)
split. (* sépare le but en deux sous-buts *)
− destruct b1. (* raisonnement par cas *)
+ reflexivity. (* true = true *)
+ simpl in H. discriminate H. (* false <> true *)
− destruct b2. (* raisonnement par cas *)
+ reflexivity. (* true = true *)
+ destruct b1; simpl in H; discriminate H.
Qed.

Remarque : Au moins pour certaines preuves qui n’utilisent que des tactiques
correspondant à des règles de logique (intuitionniste), on peut
construire un arbre de preuve.

__A vous de jouer maintenant !__

### Les prédicats inductifs

On a vu comment définir des types inductifs. De la même manière,
on peut définir des prédicats inductifs (à la Prolog).

In [ ]:
Inductive even : nat → Prop :=
| even_0 : even 0 (* 0 est pair *)
| even_SS : forall n, even n → even (S (S n)).
(* si n est pair, n+2 est pair *)

even_0 et even_SS sont les constructeurs de la définition
inductive : ce sont des axiomes. On peut s’en servir pour prouver
d’autres théorèmes.
On peut alors exprimer d’autres théorèmes

In [ ]:
Theorem multiple_2_even : forall n, even (2 ∗ n).

In [ ]:
Les prédicats inductifs (2/2)
Autre exemple.
Inductive In {A : Type} : A → list A → Prop :=
| In_first : forall (x : A) (l : list A), In x (x :: l)
| In_later : forall (x y : A) (l : list A), In x l → In x (y::l).
Lemma In_app : forall {A : Type} (x : A) (l1 l2 : list A),
In x (app l1 l2) → In x l1 ∨ In x l2.
Paramètre et indice
Dans la famille de types In A x l :
• A est un paramètre, il est global aux constructeurs
• x et l sont des indices, ils peuvent être instanciés
différemment dans chaque constructeur

In [ ]:
Preuve et prédicat inductif
Lemma even_4 : even 4.
Proof.
apply even_SS. apply even_SS. apply even_0.
Qed.
(* ou avec une syntaxe fonctionnelle*)
Theorem even_4’ : even 4.
Proof. apply (even_SS 2 (even_SS 0 even_0)).
Qed.
Theorem even_plus4 : forall n, even n → even (4 + n).
Proof.
intros n. simpl. intros Hn.
apply even_SS. apply even_SS. apply Hn.
Qed.

In [ ]:
Inversion
Supposons que l’on ait en hypothèse que n est un nombre pair
( H : even n).
Comment tirer partie de cette hypothèse ?
Si on regarde la définition de even, cela peut vouloir dire 2 choses :
• soit n est nul,
• soit il existe un p tel que n est de la forme S (S p) et p est
lui-même pair (hypothèse Hp)
Analyse de cas : inversion H as [ p Hp].|

In [ ]:
Induction
Theorem ev_minus2 : forall n,
even n → even (pred (pred n)).
Proof.
intros n H.
inversion H as [| p Hp].
− (* H = even_0 *) simpl. apply even_0.
− (* E = even_SS p Hp *) simpl. apply Hp.
Qed.
Theorem even5_nonsense :
even 5 → 2 + 2 = 9.
Proof.
intro H.
inversion H.
inversion H1.
inversion H3.
Qed.

In [ ]:
Induction
Prouvons qu’ une propriété P n est vraie pour tous les entiers pairs.
forall n, even n → P n.
On peut faire cette preuve par induction sur (even n). Pour cela il
faut montrer 2 choses qui correspondent aux 2 façons de former
des nombres pairs.
• 1er cas : n est nul (l’hypothèse a été construite avec even_0) :
il faut montrer P 0.
• 2ème cas : n est de la forme S (S k) avec k un nombre pair
(l’hypothèse a été construite avec even_SS) : Il faut alors
montrer P( S (S k)) avec l’hypothèse d’induction que la
propriété P k est vraie.

In [ ]:
Induction - exemple
Lemma ev_even : forall n,
even n → exists k, n = 2 ∗ k.
Proof.
intros n E.
induction E as [|k Hk IH].
− (* E = even_0 *)
exists 0. reflexivity.
− (* E = even_SS k Hk
with IH : exists k’, n’ = double k’ *)
destruct IH as [k’ Hk’].
.....
Qed.

In [ ]:
(** Exercice 2 *)

In [ ]:
(*
   [andb_true_iff : forall b1 b2, andb b1 b2 = true <-> b1 = true /\ b2 = true]
    Montrer [orb_true_iff], un rÃ©sultat similaire sur [orb].  
 *)

In [ ]:
Lemma plus_0_n : forall n, 0 + n = n.
Proof.
intro n. simpl. reflexivity.
Qed.

In [ ]:
Lemma plus_n_0 : forall n, n + O = n.
Proof.
induction n.
+  simpl. reflexivity.
+ simpl. rewrite IHn. reflexivity.
Qed.

In [ ]:
(** Ecrire la multiplication pour les entiers *)

In [ ]:
Fixpoint mult (n m : nat) :  nat :=
  match n with
  | O    => 0
  | S n' => m + (mult n' m)
  end.

In [ ]:
Notation "n * m" := (mult n m).

In [ ]:
(** Prouver les lemmes suivants. *)

In [ ]:
Lemma mult_0_n : forall n : nat, 0*n=0.
Proof.
intros. simpl. reflexivity.
Qed.

In [ ]:
Lemma mult_n_0 : forall n, n*0= 0.
Proof.
induction n.
+  apply mult_0_n.
+  simpl. exact IHn.
 (* ou assumption*)
Qed.

In [ ]:
Lemma plus_assoc : forall n m p,
  n+(m+p) = n+m+p.
Proof.
induction n ; intros m p.
+ simpl. reflexivity. 
+ simpl. rewrite IHn. reflexivity.
Qed.

In [ ]:
Lemma plus_n_Sm : forall n m,
  n + S m = S (n + m).
Proof.
induction n ; intro m ; simpl.
+ reflexivity.
+ rewrite IHn. reflexivity.
Qed.

In [ ]:
Lemma plus_comm : forall n m, n+m=m+n.
Proof.
induction n ; intros m0.
+ simpl. rewrite plus_n_0. reflexivity.
+ simpl. rewrite IHn. rewrite plus_n_Sm. reflexivity.
Qed.

In [ ]:
Lemma mult_n_Sm : forall n m, n*S m=n+n*m.
Proof.
induction n ; intros m0 ; simpl.
+ reflexivity.
+ rewrite IHn.
  do 2 (rewrite plus_assoc).
  rewrite plus_comm with (n:=n) (m := m0). reflexivity.
Qed.

In [ ]:
Lemma mult_comm : forall n m, n *m=m*n.
Proof.
induction n ; intros m0 ; simpl.
+ rewrite mult_n_0. reflexivity.
+ rewrite IHn. rewrite mult_n_Sm.  reflexivity.
Qed.

In [ ]:
Lemma mult_plus_distr_r : forall n m p,
  (n+m)*p = n*p+m*p.
Proof.
induction n ; intros m p; simpl.
+ reflexivity.
+ rewrite IHn. apply plus_assoc.
Qed.

In [ ]:
Lemma mult_assoc : forall n m p, n*(m*p) = (n*m)*p.
Proof.
induction n ; intros m p; simpl.
+ reflexivity.
+ rewrite IHn. rewrite mult_plus_distr_r. reflexivity. 
Qed.

In [ ]:
(* Prouver directement *)
Lemma mult_plus_distr_l : forall n m p,
  n*(m+p)=n*m+n*p.
Proof.
induction n; intros m p ; simpl.
+ reflexivity.
+ rewrite IHn. 
    rewrite plus_comm with (n:=p) (m:=n*p).
    rewrite <- plus_assoc with (n:=m) (m:=n*m) (p:=n*p+p).
    rewrite plus_comm with (n:=m) (m:=n*m + (n*p + p)).
    do 2 (rewrite plus_assoc).
    rewrite plus_comm with (n:=m) (m:=p).
    rewrite <- plus_assoc with (n:=p + m) (m:=n*m) (p:=n*p).
    rewrite plus_comm with (n:=p + m) (m:=n*m + n*p).
    rewrite plus_assoc. reflexivity.
Qed.

In [ ]:
(* Prouver en utilisant les lemmes precedents
   (mais bien sur pas [mult_plus_distr_l]) *)
Lemma mult_plus_distr_l_bis : forall n m p,
  n*(m+p)=n*m+n*p.
Proof.
intros n m p.
rewrite mult_comm with (n := n) (m := m).
rewrite mult_comm with (n := n) (m := p).
rewrite <- mult_plus_distr_r.
apply mult_comm.
Qed.

In [ ]:
End nat_bool.


(* On ferme le module.  nat_bool est de nouveau par defaut le type des entiers naturels de la bibliotheque standard de Coq. *)

(** Remarque : tous les resultats demontres dans cet exercice sont disponibles
    dans la bibliotheque [Arith], ([Require Import Arith]) *)


In [ ]:
(** * Exercice 3. *)
(** Prouver les resultats suivants. *)

In [ ]:
Lemma impl_and : forall P Q R : Prop, (P->Q->R) -> (P /\ Q -> R).
Proof.
intros P Q R Hyp1 Hyp2.
destruct Hyp2 as [Hp Hq].
apply Hyp1 ; assumption.
Qed.

In [ ]:
Lemma and_impl : forall P Q R : Prop, (P /\ Q -> R) -> (P -> Q -> R).
Proof.
intros P Q R Hyp1 H1 H2.
apply Hyp1.
split ; assumption.
Qed.

In [ ]:
Lemma and_impl_iff : forall P Q R : Prop, (P /\ Q -> R) <-> (P -> Q -> R).
Proof.
split.
+ apply and_impl.
+ apply impl_and.
Qed.

In [ ]:
Lemma impl_not : forall P : Prop, P -> ~ ~ P.
Proof.
intros P Hyp.
unfold not.
intro Hyp1.
apply Hyp1.
assumption.
Qed.

In [ ]:
Lemma modus_ponens : forall P Q : Prop, P -> (P -> Q) -> Q.
Proof.
intros P Q H1 H2.
apply H2.
assumption.
Qed.

In [ ]:
(* en utilisant [modus_ponens] *)
Lemma impl_not_bis : forall P : Prop, P -> ~ ~ P.
Proof.
intro P. unfold not.
apply modus_ponens.
Qed.

In [ ]:
(* a partir de [impl_not] *)
Lemma P_notP_contradiction : forall P : Prop, ~ (P /\ ~ P).
Proof.
intro P.
unfold not .
(*apply impl_and.
apply modus_ponens.*)
intro H1.
destruct H1 as [HH1 HH2].
apply HH2. assumption.
Qed.

In [ ]:
Lemma not_and : forall P Q : Prop, P \/ Q -> ~(~ P /\ ~Q).
Proof.
Admitted.

In [ ]:
Lemma exists_not_forall : forall P : nat -> Prop, 
(exists n, P n) ->  ~(forall n,~ (P n)).
Proof.
intros P H H1. 
destruct H as [n0 H0].
specialize (H1 n0).
contradiction.
Qed.

In [ ]:
(** * Exercice 4. *)
Module Liste. (*idem pour définir les listes san cacher celles de la 
bibliothèque standard *)

In [ ]:
Variable A : Set. (* A est ici un type abstrait *)

In [ ]:
(* on définit le type des listes dont les éléments sont de types A*)

In [ ]:
Inductive list : Set :=
| nil : list 
| cons : A -> list -> list.

In [ ]:
Check list.

In [ ]:
Notation "[]" := nil.

In [ ]:
Notation "[ x ]" := (cons x nil).

In [ ]:
Notation "[ x ; y ; .. ; z ]" := (cons x (cons y .. (cons z nil) ..)).


In [ ]:
(* Définir la fonction length qui compte les éléments d'une liste
	et la fonction app qui concatene deux listes- montrer que le
	nombre d'elements de la concatenation de deux listes est la somme
	des nombres des éléments de chacune *)

(* Définir la fonction qui renverse une liste (on utilisera la fonction prÃ©cÃ©dente app)
 et  montrer que reverse(reverse l) = l pour tout l, par induction sur l*)

In [ ]:
End Liste.

In [ ]:
Require Import List.
Require Import Arith.

In [ ]:
Check list.
Print list. (* on retrouve les listes et les naturels de la bibliothèque standard*)

In [ ]:
(** Exercice 5 *)

In [ ]:
Definition is_even n := exists p, n = 2 * p.

In [ ]:
Lemma zero_is_even : is_even 0.
Proof.
unfold is_even.
exists 0. simpl. reflexivity.
Defined.

In [ ]:
Lemma is_even_plus : forall n p, 
   is_even n -> is_even p -> is_even (n + p).
Proof.
intros n p Hn Hp.
unfold is_even in Hn.
destruct Hn as [k Hk].
destruct Hp as [t Ht].
unfold is_even.
exists (k + t).
rewrite  mult_plus_distr_l.
rewrite Hk.  rewrite Ht. reflexivity.
Qed. 

In [ ]:
(** * Exercice 5. *)

In [ ]:
(** On rappelle la définition de [even] sous forme d'un predicat inductif . *)


In [ ]:
Inductive even : nat -> Prop :=
| even_0 : even 0
| even_SS : forall n, even n -> even (S (S n)).

In [ ]:
Lemma even_4 : even 4.
Proof.
apply even_SS. apply even_SS. apply even_0.
Qed.

In [ ]:
(* ou avec une syntaxe fonctionnelle*)
Theorem even_4' : even 4.
Proof. exact (even_SS 2 (even_SS 0 even_0)).
Qed.

In [ ]:
Theorem even_plus4 : forall n, even n -> even (4 + n).
Proof.
  intros n. simpl. intros Hn.
  apply even_SS. apply even_SS. assumption.
Qed.

In [ ]:
Theorem ev_minus2 : forall n,
  even n -> even (pred (pred n)).
Proof.
  intros n H.
  inversion H  as [| p Hp].
  - (* H = even_0 *) simpl. apply even_0.
  - (* H = even_SS p Hp *) simpl. exact Hp (* ou assumption*).
Qed.

In [ ]:
Theorem even5_nonsense :
  even 5 -> 2 + 2 = 9.
Proof.
  intro H.
  inversion H. 
  inversion H1.
  inversion H3.
Qed.

In [ ]:
Lemma ev_even : forall n,
  even n -> exists k, n = 2 * k.
Proof.
  intros n E.
  induction E as [|k Hk IH].
  - (* E = even_0 *)
    exists 0. reflexivity.
  - (* E = even_SS k Hk
       with IH : exists k', n' = double k' *)
    destruct IH as [k' Hk'].
    rewrite Hk'. exists (S k'). simpl.
    Search plus.
    rewrite <- plus_n_Sm.
    replace (k' + 0) with k'. reflexivity.
    apply plus_n_O.
Qed.

In [ ]:
(** Prouver les lemmes suivants. *)

Lemma even_n_or_Sn : forall n, even n \/ even (S n).
Proof.
Admitted.

In [ ]:
Lemma even_n_not_Sn : forall n, even n -> ~ even (S n).
Proof.
Admitted.

In [ ]:
Lemma even_dec : forall n, even n \/ ~ even n.
Proof.
Admitted.

In [ ]:
(** Induction de 2 en 2. *)

In [ ]:
Lemma two_steps_induction : forall P : nat -> Prop, P 0 -> P 1 ->
  (forall n, P n -> P (S (S n))) ->
  forall n, P n.
Proof.
  intros P H0 H1 IHn.
  assert (forall n, P n /\ P (S n)) as Hn.
Admitted.

In [ ]:
(** DÃ©finir [evenb : nat -> bool] qui teste si un entier est pair. *)


In [ ]:
Fixpoint evenb (n:nat) : bool.
Admitted.

In [ ]:
(** Prouver les lemmes suivants. Si vous avez besoin de résultats sur les
    booléens, vous pouvez les trouver dans [Bool]
    (utiliser [Require Import Bool.]).
*)


In [ ]:
Lemma evenb_correct : forall n, evenb n = true -> even n.
Proof.
Admitted.

In [ ]:
Lemma evenb_complete : forall n, even n -> evenb n = true.
Proof.
Admitted.

In [ ]:
Definition evenb_spec : forall n, evenb n = true <-> even n.
Proof.
Admitted.

In [ ]:
(** Prouver [even_dec_bis : forall n, even n \/ ~ even n]
    en utilisant [evenb_spec]. *)

In [ ]:
Lemma even_dec_bis : forall n, even n \/ ~ even n.
Proof.
Admitted.

In [ ]:
(** Exercice 6 *)
(** retour des listes *)

In [ ]:
Variable A : Type.

In [ ]:
Inductive mem : A -> list A -> Prop :=
  mem_head : forall x l, mem x (cons x l)
| mem_tail : forall x y l, 
   mem x l -> mem x (cons y l).


In [ ]:
Check app.

In [ ]:
Print app.

In [ ]:
Lemma app_left : forall l1 l2 x,
mem x l1 -> mem x (app l1 l2).
Proof.
Admitted.

In [ ]:
Lemma app_or : forall l1 l2 x, mem x (app l1 l2) -> mem x l1 \/ mem x l2.
Proof.
Admitted.

__Tactiques pour manipuler les types inductifs__

• destruct x pour une analyse par cas de x

• discriminate pour utiliser la disjonction des constructeurs

• injection H pour utiliser l’injection des constructeurs dans H

• inversion H ' destruct H, mais plus pour les prédicats

• induction H ' destruct H avec hypothèse d’induction

• constructor pour appliquer un constructeur du type de la
conclusion

__Pour manipuler la logique__

• intros pour introduire les variables quantifiées universellement

• exists x pour donner la valeur témoin attendue

• split pour détruire un but en plusieurs sous-buts

• left, right pour choisir une branche d’une disjonction

• reflexivity pour utiliser la réflexivité de l’égalité

• symmetry pour utiliser la symétrie de l’égalité

• contradiction si les hypothèses contiennent False ou P et ~P

__Autres__

• exact H lorsque l’hypothèse H et le but actuel sont identiques

• assumption pour chercher dans les hypothèses

• apply H pour appliquer H dont la conséquence est la
conclusion actuelle

• rewrite [← ] H pour récrire en utilisant l’égalité H

• replace x with y pour remplacer x par y en prouvant x=y

• subst x pour éliminer x en utilisant les égalités disponibles

• assert H as Ha pour introduire un lemme intermédiaire H
pendant la preuve

• specialize (H x) pour appliquer partiellement une hypothèse

• pose proof lem as H pour ajouter un théorème aux hypothèses

• simpl pour simplifier des termes

• unfold f pour remplacer f par sa définition